In [6]:
import numpy as np

DEFAULT = None
CPU = 'cpu'
np.random.seed(42)


class Tensor:
    def __init__(self, array:np.ndarray, _children=(), _op='', label=''):
        self.shape = array.shape
        self.array = array
        self.grad = np.zeros(self.shape, dtype=np.float32)
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
    def __repr__(self) -> str:
        # return f"Value(shape={self.array.shape}, grad={self.grad.shape})"
        return self.array.__repr__()
    def __add__(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.array + other.array, (self, other), '+')
        def _backward():
            self.grad += np.ones_like(self.array) * out.grad
            if other.shape != out.shape:
                other.grad += (np.ones_like(other.array) * out.grad).sum(axis=0)
            else:
                other.grad += np.ones_like(other.array) * out.grad
            # self.grad += 1.0 * out.grad
            # other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    def __radd__(self, other):
        return self + other
    def __neg__(self): return self * (-1)
    def __sub__(self, other):
        return self + (-other)
    def __rsub__(self, other):
        return other + (-self)
    def __mul__(self, other):
        other = other if isinstance(other, np.ndarray) else -np.ones_like(self.array)
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.array * other.array, (self, other), '*')
        def _backward():
            self.grad += other.array * out.grad
            other.grad += self.array * out.grad
        out._backward = _backward
        return out
    def __rmul__(self, other): return self * other
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supports int/float powers for now"
        out = Tensor(self.array ** other, (self, ), f'**{other}')
        def _backward():
            self.grad += other * (self.array ** (other-1)) * out.grad
        out._backward = _backward
        return out
    
    def dot(self, other):
        other = other if isinstance(other, Tensor) else Tensor(other)
        assert len(other.shape) == 1, "support only one dimension use mm instead"
        ...
    def mm(self, other):
        assert len(other.shape) == 2, "not support higher dimensions then matrix"
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.array.dot(other.array), (self, other), 'mm')
        def _backward():
            # self.grad += np.tile(other.array.sum(axis=-1), (self.shape[-2], 1))# * out.grad
            self.grad += np.dot(out.grad, other.array.T)
            # other.grad += np.tile(self.array.sum(axis=-2), (other.shape[-1], 1)).T #* out.grad
            other.grad = np.dot(self.array.T, out.grad)
        out._backward = _backward
        return out
    # def matmul
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited: 
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = np.ones(self.shape)
        for node in reversed(topo):
            node._backward()
    def exp(self):
        x = self.array
        out = Tensor(np.exp(x), (self, ), 'exp')
        def _backward():
            self.grad += out.array * out.grad
        out._backward = _backward
        return out
    def mean(self, axis=...):
        x = self.array
        out = Tensor(x.mean(axis=axis), (self, ), 'mean')
        def _backward():
            self.grad += out.grad/self.shape[0]
        out._backward = _backward
        return out
    def sum(self, axis=...):
        x = self.array
        out = Tensor(x.sum(axis=axis), (self, ), 'sum')
        def _backward():
            self.grad += out.grad
        out._backward = _backward
        return out
    def step(self, lr): 
        lr = np.array([lr], dtype=np.float32)
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited: 
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        for node in reversed(topo):
            node.array -= lr * node.grad
    def zero_grad(self): 
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited: 
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        for node in reversed(topo):
            node.grad *= 0
    def item(self):
        assert len(self.shape) == 1 and self.shape[0] == 1
        return self.array[0]
    
class Loss: ...
class MSELoss:
    def __init__(self):
        ...
    def __call__(self, y:Tensor, y_real:Tensor, axis=0):
        return ((y - y_real)**2).mean(axis=axis)

class Optimizer: ...
class Adam: ...
class SGD: ... # StochasticGradDescent


class Activation: ...
class ReLU(Activation): ...

class Sigmoid(Activation): 
    def __init__(self): 
        self.grad = None
    def __call__(self, x:Tensor):
        return self.forward(x)
    def forward(self, x:Tensor):
        self.grad = self.backward(x)
        # return 1/(1+np.exp(-x))
        return (Tensor(np.ones_like(x.array))+(-x).exp())**(-1)
    
    def backward(self, x):
        ...
        # return x * (1-x)

class Module:
    def __init__(self): ...
    def __call__(self): ...
class Linear:
    def __init__(self, in_features, out_features, bias=True, device=DEFAULT, dtype=DEFAULT):
        self.dtype = dtype if dtype is not None else np.float32
        self.device = device if device is not None else CPU
        
        self.W = Tensor(np.random.uniform(-1, 1, (in_features, out_features)).astype(self.dtype)) # no need to transpose the tensors as i am initialized here with Transpose shape
        # self.W = np.random.random((in_features, out_features)).astype(self.dtype)
        self.B = Tensor(np.zeros(out_features, dtype=self.dtype))
    def __call__(self, X:Tensor):
        return self.forward(X)
    def forward(self, x:Tensor):
        return x.mm(self.W) + self.B
        # return np.dot(x, self.W) + self.B
    def backward(self, x):
        ...

In [7]:
class TestModel:
    def __init__(self) -> None:
        self.fc1 = Linear(30, 10)
        self.fn1 = Sigmoid()
        self.fc2 = Linear(10, 1)
        self.fn2 = Sigmoid()
    def __call__(self, X):
        return self.forward(X)
    def forward(self, x):
        x = self.fc1(x)
        x = self.fn1(x)
        x = self.fc2(x)
        # return x
        return self.fn2(x)

In [8]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 1: Load real data (Breast Cancer Wisconsin dataset)
data = load_breast_cancer()
X, y_real = data.data.astype(np.float32), data.target.astype(np.float32)
y_real = y_real.reshape(-1, 1)

# Step 2: Preprocess data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_real, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

model = TestModel()
criterion = MSELoss()
for epoch in range(5000):
    y = model(X_train)    
    loss = criterion(y, y_train)
    if epoch % 100 == 0:
        print(f"EPOCH: {epoch} | LOSS: {loss.item():.4f}")
    loss.backward()
    loss.step(0.003)
    loss.zero_grad()

    
# Step 5: Evaluate model
outputs = model(X_test)
preds = outputs.array

accuracy = (np.round(preds) == y_test.array).mean()
print(f'Accuracy on test data: {accuracy:.4f}')

EPOCH: 0 | LOSS: 0.1732
EPOCH: 100 | LOSS: 0.1672
EPOCH: 200 | LOSS: 0.1614
EPOCH: 300 | LOSS: 0.1561
EPOCH: 400 | LOSS: 0.1510
EPOCH: 500 | LOSS: 0.1463
EPOCH: 600 | LOSS: 0.1418
EPOCH: 700 | LOSS: 0.1376
EPOCH: 800 | LOSS: 0.1336
EPOCH: 900 | LOSS: 0.1299
EPOCH: 1000 | LOSS: 0.1263
EPOCH: 1100 | LOSS: 0.1229
EPOCH: 1200 | LOSS: 0.1197
EPOCH: 1300 | LOSS: 0.1167
EPOCH: 1400 | LOSS: 0.1138
EPOCH: 1500 | LOSS: 0.1111
EPOCH: 1600 | LOSS: 0.1085
EPOCH: 1700 | LOSS: 0.1060
EPOCH: 1800 | LOSS: 0.1037
EPOCH: 1900 | LOSS: 0.1014
EPOCH: 2000 | LOSS: 0.0993
EPOCH: 2100 | LOSS: 0.0972
EPOCH: 2200 | LOSS: 0.0952
EPOCH: 2300 | LOSS: 0.0933
EPOCH: 2400 | LOSS: 0.0915
EPOCH: 2500 | LOSS: 0.0898
EPOCH: 2600 | LOSS: 0.0881
EPOCH: 2700 | LOSS: 0.0865
EPOCH: 2800 | LOSS: 0.0849
EPOCH: 2900 | LOSS: 0.0834
EPOCH: 3000 | LOSS: 0.0820
EPOCH: 3100 | LOSS: 0.0806
EPOCH: 3200 | LOSS: 0.0792
EPOCH: 3300 | LOSS: 0.0779
EPOCH: 3400 | LOSS: 0.0767
EPOCH: 3500 | LOSS: 0.0755
EPOCH: 3600 | LOSS: 0.0743
EPOCH: 3700 |